# CompuOpti

In [409]:
import os
import json

import numpy as np
import gurobipy as grb
from gurobipy import GRB

In [410]:
INF = 2 ** 64

In [411]:
model = grb.Model()

In [412]:
instance = "toy_instance.json"

file = os.path.join("instances", instance)
data = json.load(open(file, "r"))

## Modeling

### Parameters of the problem

Define the length of indices

In [413]:
# Number of workers
worker_length = len(data["staff"])

# Number of jobs
job_length = len(data["jobs"])

# Number of skills
skill_length = len(data["qualifications"])

# Number of days
day_length = data["horizon"]

Define jobs parameters

In [414]:
gains_job = np.array([job["gain"] for job in data["jobs"]])
penalties_job = np.array([job["daily_penalty"] for job in data["jobs"]])
due_dates_job = np.array([job["due_date"] for job in data["jobs"]])
work_days_job_skill = np.array([
    [
        job["working_days_per_qualification"][skill] if skill in job["working_days_per_qualification"] else 0
        for skill in data["qualifications"]
    ]
    for job in data["jobs"]
])

Define staff parameters

In [415]:
qualifications_worker_skill = np.array([
    [
        1 if skill in worker["qualifications"] else 0
        for skill in data["qualifications"]
    ]
    for worker in data["staff"]
])
vacations_worker_day = np.array([
    [
        1 if 1 + day in worker["vacations"] else 0
        for day in range(day_length)
    ]
    for worker in data["staff"]
])

### Decision variables

Variable to model the full solution

In [416]:
# 4-D array of binary variables : 1 if a worker is assigned to a certain project for a certain skill on a certain day, else 0
works_worker_job_skill_day = model.addVars(worker_length, job_length, skill_length, day_length, vtype=GRB.BINARY, name="work")

Variables to compute the total gain

In [417]:
is_realized_job = model.addVars(job_length, vtype=GRB.BINARY, name="is_realized") # 1 if a job is realized, else 0

Variable to compute the maximum duration (and the penalties)

In [418]:
is_worked_job_day = model.addVars(job_length, day_length, vtype=GRB.BINARY, name="is_worked") # 1 if a certain day is a work day for a certain job, else 0
max_duration = model.addVar(vtype=GRB.INTEGER, name="max_duration") # Integer that represents the maximum duration for any job

Variables to compute the maximum assignement

In [419]:
is_assigned_worker_job = model.addVars(worker_length, job_length, vtype=GRB.BINARY, name="is_assigned") # 1 if a certain worker is assigned on a certain job, else 0
max_assigned = model.addVar(vtype=GRB.INTEGER, name="max_assigned") # Integer that represents the maximum number of assigned jobs for any worker

### Constraints

#### Time Table Constraints

Define the constraints of the planning problem itself.

- Worker qualification constraint

In [420]:
model.addConstrs(
    (
        works_worker_job_skill_day[worker, job, skill, day] <= qualifications_worker_skill[worker, skill]
        for worker in range(worker_length)
        for job in range(job_length)
        for skill in range(skill_length)
        for day in range(day_length)
    ),
    name="qualification",
)

{(0, 0, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 2): <gurobi.Constr *Awaiting Model Up

- Uniqueness of the daily assignement : *done with the vacation constraint (see below)*

- Vacation constraint

In [421]:
model.addConstrs(
    (
        grb.quicksum(works_worker_job_skill_day[worker, job, skill, day] for job in range(job_length) for skill in range(skill_length)) \
            <= 1 - vacations_worker_day[worker, day]
        for worker in range(worker_length)
        for day in range(day_length)
    ),
    name="vacation",
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>}

- Job coverage constraint

In [422]:
model.addConstrs(
    (
        grb.quicksum(works_worker_job_skill_day[worker, job, skill, day] for worker in range(worker_length) for day in range(day_length)) \
            == is_realized_job[job] * work_days_job_skill[job, skill]
        for job in range(job_length)
        for skill in range(skill_length)
    ),
    name="job_coverage",
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>}

- Uniqueness of the realized projects : *done with realized_job either 0 or 1*

#### Variable Constraints

Define the constraints that are related to the definition of additional variables.

- Is realized : *done with job coverage constraint*

- Is worked

In [423]:
# exists_worker_skill works == 1 => is_worked == 1
model.addConstrs(
    (
        works_worker_job_skill_day[worker, job, skill, day] <= is_worked_job_day[job, day]
        for worker in range(worker_length)
        for job in range(job_length)
        for skill in range(skill_length)
        for day in range(day_length)
    ),
    name="is_worked_job_day",
)
# forall_worker_skill works == 0 => is_worked == 0
model.addConstrs(
    (
        is_worked_job_day[job, day] <= \
            grb.quicksum(
                works_worker_job_skill_day[worker, job, skill, day]
                for worker in range(worker_length)
                for skill in range(skill_length)
            )
        for job in range(job_length)
        for day in range(day_length)
    ),
    name="is_worked_job_day_bis",
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,


- Maximum duration

In [424]:
model.addConstrs(
    (
        grb.quicksum(is_worked_job_day[job, day] for day in range(day_length)) \
            <= max_duration
        for job in range(job_length)
    ),
    name="max_duration",
)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

- Is assigned

In [425]:
# exists_skill_day works == 1 => is_assigned == 1
model.addConstrs(
    (
        works_worker_job_skill_day[worker, job, skill, day] <= is_assigned_worker_job[worker, job]
        for worker in range(worker_length)
        for job in range(job_length)
        for skill in range(skill_length)
        for day in range(day_length)
    ),
    name="is_assigned_worker_job",
)
# forall_skill_day works == 0 => is_assigned == 0
model.addConstrs(
    (
        is_assigned_worker_job[worker, job] <= \
            grb.quicksum(
                works_worker_job_skill_day[worker, job, skill, day]
                for skill in range(skill_length)
                for day in range(day_length)
            )
        for worker in range(worker_length)
        for job in range(job_length)
    ),
    name="is_assigned_worker_job_bis",
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>}

- Maximum assigned

In [426]:
model.addConstrs(
    (
        grb.quicksum(is_assigned_worker_job[worker, job] for job in range(job_length)) \
            <= max_assigned
        for worker in range(worker_length)
    ),
    name="max_assigned",
)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

### Objectives

In [427]:
# Add primary objective
model.setObjective(
  grb.quicksum(
    gains_job[job] * is_realized_job[job] \
      - penalties_job[job] * grb.quicksum(
        is_worked_job_day[job, day]
        for day in range(due_dates_job[job], day_length)
      )
    for job in range(job_length)
  ),
  sense=GRB.MAXIMIZE,
)

# # Add multi-objective functions
# model.ModelSense = GRB.MINIMIZE
# model.setObjectiveN(
#   max_assigned,
#   1,
#   priority=2,
# )
# model.setObjectiveN(
#   max_duration,
#   2,
#   priority=1,
# )

## Optimization

In [428]:
model.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 753 rows, 272 columns and 2123 nonzeros
Model fingerprint: 0x62922262
Variable types: 0 continuous, 272 integer (270 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 617 rows and 148 columns
Presolve time: 0.01s
Presolved: 136 rows, 124 columns, 611 nonzeros
Variable types: 0 continuous, 124 integer (124 binary)

Root relaxation: objective 7.200000e+01, 154 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 

In [429]:
# Get optimization status
print(model.Status == GRB.OPTIMAL, model.Status == GRB.TIME_LIMIT, model.Status == GRB.INFEASIBLE)

True False False


In [430]:
# Get objective value
objective_value = model.objVal
print(objective_value)

67.0


In [432]:
import pandas as pd
import matplotlib as mpl

In [433]:
qualifications = data["qualifications"]
names = [worker["name"] for worker in data["staff"]]
jobs = [job["name"] for job in data["jobs"]]

In [434]:
# Give a hex color to each job
cmap = mpl.cm.get_cmap('hsv', int(len(jobs)*1.2))
job_colors = [mpl.colors.rgb2hex(cmap(i)[:3]) for i in range(cmap.N)][:len(jobs)]

In [435]:
def disply_worker_skills():
    res = pd.DataFrame(
        [
            [
                str([q for (r, q) in zip(row_q, qualifications) if r == 1]).replace("'", ""),
                str([d for (r, d) in zip(row_d, range(day_length)) if r == 1]),
            ]
            for row_q, row_d in zip(qualifications_worker_skill, vacations_worker_day)],
        index=names,
        columns=["qualifications", "vacations"]
    )
    return res
    

In [436]:
def highlight_cols(col):
    return [f"color:black;background-color: {color}" for color in job_colors]

def display_work_days():
    res = pd.DataFrame(work_days_job_skill, index=jobs, columns=qualifications)
    res["due date"] = pd.Series(due_dates_job, index=jobs)
    res["gain"] = pd.Series(gains_job, index=jobs)
    res["penalty"] = pd.Series(penalties_job, index=jobs)
    res = res.style.apply(highlight_cols, axis=0)
    return res

In [437]:
def find_task(tab, worker, day):
    for job in range(job_length):
        for skill in range(skill_length):
            if tab[worker, job, skill, day].x == 1:
                return job, qualifications[skill]
    return None

def color_cells(x, df):
        df = df.applymap(lambda val: job_colors[val[0]] if val is not None else "")
        df = df.applymap(lambda color: f"color:{'' if color == '' else 'black'};background-color: {color}")
        return df

def display_time_table():
    data = [
        [
            find_task(works_worker_job_skill_day, worker, day)
            for day in range(day_length)
        ]
        for worker in range(worker_length)
    ]
    df = pd.DataFrame(
        data,
        index=names,
        columns=range(day_length)
    )
    res = df.applymap(lambda x: x[1] if x is not None else None)
    res = res.style.apply(lambda x: color_cells(x, df), axis=None)
    return res

In [438]:
disply_worker_skills()

,qualifications,vacations
Olivia,"[A, B, C]",[]
Liam,"[A, B]",[0]
Emma,[C],[1]


In [439]:
display_work_days()

,A,B,C,due date,gain,penalty
Job1,1,1,1,3,20,3
Job2,1,2,0,3,15,3
Job3,1,0,2,4,15,3
Job4,0,2,1,3,20,3
Job5,0,0,2,5,10,3


In [440]:
display_time_table()

,0,1,2,3,4
Olivia,B,B,A,C,B
Liam,None,A,B,A,B
Emma,C,None,C,C,None
